# Introduction

## Objective

Use Llama3, Langchain and FAISS to create a Retrieval Augmented Generation (RAG) system. This will allow us to ask questions about our ICD10 document (that was not included in the training data), without fine-tunning the Large Language Model (LLM).
When using RAG, if you are given a question, you first do a retrieval step to fetch any relevant documents from a special database, a vector database where these documents were indexed. 

## Definitions

* LLM - Large Language Model  
* Llama3 - LLM from Meta 
* Langchain - a framework designed to simplify the creation of applications using LLMs
* Vector database - a database that organizes data through high-dimmensional vectors  
* FAISS - vector database  
* RAG - Retrieval Augmented Generation (see below more details about RAGs)


## What is a Retrieval Augmented Generation (RAG) system?

Large Language Models (LLMs) has proven their ability to understand context and provide accurate answers to various NLP tasks, including summarization, Q&A, when prompted. While being able to provide very good answers to questions about information that they were trained with, they tend to hallucinate when the topic is about information that they do "not know", i.e. was not included in their training data. Retrieval Augmented Generation combines external resources with LLMs. The main two components of a RAG are therefore a retriever and a generator.  
 
The retriever part can be described as a system that is able to encode our data so that can be easily retrieved the relevant parts of it upon queriying it. The encoding is done using text embeddings, i.e. a model trained to create a vector representation of the information. The best option for implementing a retriever is a vector database. As vector database, there are multiple options, both open source or commercial products. Few examples are ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Our option in this Notebook will be a local instance of ChromaDB (persistent).

# Installations, imports, utils

In [ ]:
!pip install -q langchain
!pip install -q langchain_core
!pip install -q langchain_community
!pip install -q torch
!pip install -q gradio
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q colab-xterm
!pip install -q chromadb
!pip install -q ollama
!pip install -q langchain-text-splitters

In [ ]:
import torch
import ollama
from langchain.llms import HuggingFacePipeline
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
import numpy as np
import pandas as pd
import csv
import re

# Initialize model, tokenizer, query pipeline

In [ ]:
# Read the input from the file
with open('icd10cm-codes-2024.txt', 'r') as file:
    lines = file.readlines()

# Process each line and format the output
formatted_lines = []
# Extract descriptions
descriptions = []
pattern = re.compile(r'^([A-Za-z0-9]+)\s+(.*)')

for line in lines:
    match = pattern.match(line)
    if match:
        code = match.group(1)
        description = match.group(2)
        formatted_line = f"{code} is the icd code for {description.strip()}"
        formatted_lines.append(formatted_line)
        descriptions.append(description.strip())

# Write the formatted output to a new file
with open('icd10-document.txt', 'w') as file:
    for formatted_line in formatted_lines:
        file.write(formatted_line + '\n')

# Write the descriptions to a CSV file
with open('diagnosis-descriptions.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Descriptions'])  # Write the header
    for description in descriptions:
        writer.writerow([description])

In [ ]:
# This is a long document we can split up.
with open("./dummy_data.txt") as f:
    each_icd10 = f.read()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n"],
    chunk_size=1,
    chunk_overlap=0
)

## Ingestion of data using Text Loder

We will ingest the newest ICD10 codes from 2024 year.

In [ ]:
# 1. Load the data
loader = TextLoader('./dummy_data.txt')
splits = loader.load_and_split(text_splitter)

In [ ]:
splits

In [ ]:
print(len(splits))

In [ ]:
# 2. Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="llama3")

## Split data in chunks

We split data in chunks using a recursive character text splitter.

In [ ]:
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [ ]:
vectorstore.save_local("faiss_index")

## Creating Embeddings and Storing in Vector Store

In [ ]:
saved_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [ ]:
# 3. Call Ollama Llama3 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Retrieval Augmented Generation

In [ ]:
# 4. RAG Setup
retriever = saved_db.as_retriever()
def combine_lines(lines):
    return "\n".join(line.page_content for line in lines)

def rag_chain(question):
    retrieved_lines = retriever.invoke(question)
    formatted_context = combine_lines(retrieved_lines)
    return ollama_llm(question, formatted_context)

## Test the query pipeline

We test the pipeline with a query about ICD10 codes for diseases.

In [ ]:
# 5. Use the RAG App
result = rag_chain("icd codes for amebiasis?")
print(result)

# References  

[1] Murtuza Kazmi, Using LLaMA 2.0, FAISS and LangChain for Question-Answering on Your Own Data, https://medium.com/@murtuza753/using-llama-2-0-faiss-and-langchain-for-question-answering-on-your-own-data-682241488476  

[2] Patrick Lewis, Ethan Perez, et. al., Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks, https://browse.arxiv.org/pdf/2005.11401.pdf 

[3] Minhajul Hoque, Retrieval Augmented Generation: Grounding AI Responses in Factual Data, https://medium.com/@minh.hoque/retrieval-augmented-generation-grounding-ai-responses-in-factual-data-b7855c059322  

[4] Fangrui Liu	, Discover the Performance Gain with Retrieval Augmented Generation, https://thenewstack.io/discover-the-performance-gain-with-retrieval-augmented-generation/

[5] Andrew, How to use Retrieval-Augmented Generation (RAG) with Llama 2, https://agi-sphere.com/retrieval-augmented-generation-llama2/   

[6] Yogendra Sisodia, Retrieval Augmented Generation Using Llama2 And Falcon, https://medium.com/@scholarly360/retrieval-augmented-generation-using-llama2-and-falcon-ed26c7b14670   

